Beacuse this is my very first attempt into **Question Answering System**, this notebook is not the original rather i have only edited some parts and added few codes. The original notebook can be found <a href='https://www.kaggle.com/jonathanbesomi/a-qa-model-to-answer-them-all'>here</a>  
There are many ways and frameworks for building QA system, i have used **DistilBert** for this task here. see <a href='https://programmerbackpack.com/bert-nlp-using-distilbert-to-build-a-question-answering-system/'>this</a> article for detail explanations or any articles in your convienent for this topic.  
Also i have used earlier versionn of transformer library as latest version didnot worked for me(i could not figure out the way around).

In [ ]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 22.8MB/s 
     |████████████████████████████████| 133kB 51.5MB/s 
     |████████████████████████████████| 901kB 45.1MB/s 
     |████████████████████████████████| 1.2MB 46.0MB/s 
     |████████████████████████████████| 5.6MB 51.7MB/s 
     |████████████████████████████████| 7.5MB 32.8MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
ERROR: botocore 1.20.73 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
!pip install rank_bm25 -q

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import os 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import re
import json
import pickle
import glob
import torch
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
from rank_bm25 import BM25Okapi # Search engine

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
#import warnings
#warnings.filterwarnings("ignore")

pd.set_option('mode.chained_assignment',None)

In [ ]:
# use this in colab to access project directory
#os.chdir('/content/drive/MyDrive/programs/Datasets/CORD-19 Challenge') # project folder

In [ ]:
#load data
data = pd.read_csv('data/metadata.csv', low_memory=False)
print(data.shape)
data.head(3)

(536817, 19)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [ ]:
#data.isna().sum()

In [ ]:
data = data.dropna(subset=['abstract','title'])
data.drop_duplicates(subset=['title','abstract'],inplace=True)
data = data.reset_index()
data.shape

(363206, 20)

In [ ]:
data.isna().sum()

index                    0
cord_uid                 0
sha                 225757
source_x                 0
title                    0
doi                 151942
pmcid               233729
pubmed_id           179902
license                  0
abstract                 0
publish_time           197
authors               2903
journal              29360
mag_id              363206
who_covidence_id    226900
arxiv_id            356338
pdf_json_files      225757
pmc_json_files      255211
url                 134722
s2_id                31895
dtype: int64

In [ ]:
stop_words = set(stopwords.words('english'))
# function to preprocess and remove stopwords
def preprocess(text):
  no_stops = []
  sent = text.lower()
  sent = re.sub(r"[^a-zA-Z]"," ",sent)    # removes all characters except alphabets
  sent = re.sub(r'\s+', ' ', sent)       # removes extra white spaces
  #sent = str(text)
  for w in sent.split():
    if not w in stop_words:
      if len(w)>1 and not w.isnumeric():
        no_stops.append(w)
  return no_stops

In [ ]:
# nothing special here. we just take two columns we need; title and abstract than we join them and do preprocessing and find bm25 
def init(data):
  df = data.copy()
  df = data[['title','abstract']]
  df['title_abstract'] = data.abstract + " " + df.title
  df['title_abstract'] = df['title_abstract'].apply(lambda x: preprocess(x))
  bm25 = BM25Okapi(df.title_abstract.tolist())
  return bm25, df

#bm25, df_new = init(data)

In [ ]:
# for a given query find its best contexts. here we will consider top 10 contexts from dataframe of over 300k absctracts for each query
def searchContext(query,bm25, df_new, num):
  """
  Return top `num` results that better match the query
  """
  # obtain scores
  search_terms = preprocess(query) 
  doc_scores = bm25.get_scores(search_terms)
        
  # sort by scores
  ind = np.argsort(doc_scores)[::-1][:num] 
        
  # select top results and returns
  results = df_new.iloc[ind][df_new.columns]
  results['score'] = doc_scores[ind]
  results = results[results.score > 0]
  return results.reset_index()

In [ ]:
# if gpu is available use that else use cpu
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = model.to(torch_device)

# given a questions and corresponding contexts, find the answer.
def AnswerQuestions(question, context):
  # anser question given question and context
  encoded_dict = tokenizer.encode_plus(
                        question, context,
                        add_special_tokens = True,
                        max_length = 256,
                        pad_to_max_length = True,
                        return_tensors = 'pt'
                   )
    
  input_ids = encoded_dict['input_ids'].to(torch_device)
  token_type_ids = encoded_dict['token_type_ids'].to(torch_device)
    
  start_scores, end_scores = model(input_ids, token_type_ids=token_type_ids)

  all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
    
  answer = tokenizer.convert_tokens_to_string(all_tokens[start_index:end_index+1])
  answer = answer.replace('[CLS]', '')
  return answer

In [ ]:
bm25, df = init(data)
with open('saved/bm25.pkl','wb') as file:
  pickle.dump(bm25,file)

In [ ]:
num_contexts = 10
def obtain_all_contexts(query,num):
  context_df = searchContext(query,bm25,df, num_contexts)
  return context_df.abstract.to_list()

def obtain_all_answers(query, all_contexts):
  all_Answers = []
  for context in all_contexts:
    all_Answers.append(AnswerQuestions(query,context))
  return all_Answers

def getResult(query):
  all_contexts = obtain_all_contexts(query,num_contexts)
  all_answers = obtain_all_answers(query,all_contexts)
  return all_answers


In [ ]:
def answers(tasks):
  ALL_ANSWERS = []
  All_Questions = []
  for task in tasks:
    for question in task:
      Answers = getResult(question)
      Answers = list(set(Answers))
      ALL_ANSWERS.append(Answers)
      All_Questions.append(question)
      print('Query:',question)
      print('Answers: ')
      for i, ans in enumerate(Answers):
        i = i+1
        print(str(i)+')',ans)
      print('\n******************************************************************************************************************\n')  

In [ ]:
# questions to be annswered

task1 = [
         "Is the virus transmitted by aerisol, droplets, food, close contact, fecal matter, or water?",
         "How long is the incubation period for the virus?",
         "Can the virus be transmitted asymptomatically or during the incubation period?",
         "How does weather, heat, and humidity affect the tramsmission of 2019-nCoV?",
         "How long can the 2019-nCoV virus remain viable on common surfaces?"
        ]
    
task2 =     [
                  "What risk factors contribute to the severity of 2019-nCoV?",
                  "How does hypertension affect patients?",
                  "How does heart disease affect patients?",
                  "How does copd affect patients?",
                  "How does smoking affect patients?",
                  "How does pregnancy affect patients?",
                  "What is the fatality rate of 2019-nCoV?",
                  "What public health policies prevent or control the spread of 2019-nCoV?"
              ]

task3 =      [
                  "Can animals transmit 2019-nCoV?",
                  "What animal did 2019-nCoV come from?",
                  "What real-time genomic tracking tools exist?",
                  "What geographic variations are there in the genome of 2019-nCoV?",
                  "What effors are being done in asia to prevent further outbreaks?"
              ]

task4 =       [
                  "What drugs or therapies are being investigated?",
                  "Are anti-inflammatory drugs recommended?"
              ]

task5 =       [
                  "Which non-pharmaceutical interventions limit tramsission?",
                  "What are most important barriers to compliance?"
              ]

task6 =       [
                  "How does extracorporeal membrane oxygenation affect 2019-nCoV patients?",
                  "What telemedicine and cybercare methods are most effective?",
                  "How is artificial intelligence being used in real time health delivery?",
                  "What adjunctive or supportive methods can help patients?"
              ]

task7 =       [
                  "What diagnostic tests (tools) exist or are being developed to detect 2019-nCoV?"
              ]

task8 =       [
                  "What is the immune system response to 2019-nCoV?",
                  "Can personal protective equipment prevent the transmission of 2019-nCoV?",
                  "Can 2019-nCoV infect patients a second time?"
              ]

In [ ]:
%%time
tasks = [task1, task2, task3, task4, task5, task6, task7, task8]
result = answers(tasks)

Query: Is the virus transmitted by aerisol, droplets, food, close contact, fecal matter, or water?
Answers: 
1) it can be transmitted via the fecal - oral / ocular route
2) air
3) viruses are often transmitted via food and the environment
4) virus transmission by the airborne route may be both direct and indirect inhalation of infectious droplets or through contact with contaminated fomites
5) either by person - to - person contact or by ingestion of contaminated food or water
6) the virus has high transmissibility and is rapidly transmitted to people through close contact and droplets from coughing , sneezing and talking loudly , as well as through contact with contaminated objects
7) droplets

******************************************************************************************************************

Query: How long is the incubation period for the virus?
Answers: 
1) 5 . 8 days
2) 2 - 14 days
3) before the onset of symptoms
4) 2 days
5) 6 . 2
6) 5 . 894 days
7) long
8) 2 – 14

If we see properly above answers to the query we find that some answers are vague and confusing while others are straightforward and concise. To better understand those vague answers we need the context on which they appear.

In [ ]:
with open('saved/bm25.pkl','rb') as file:
  bm25 = pickle.load(file)

In [ ]:
query = "Is the virus transmitted by aerisol, droplets, food, close contact, fecal matter, or water?"
answers = getResult(query)
contexts = obtain_all_contexts(query,10)
#article_title = contexts_df.title.to_list()
#context = contexts_df.abstract.to_list()

In [ ]:
#answers = list(set(answers))
answers

['either by person - to - person contact or by ingestion of contaminated food or water',
 'virus transmission by the airborne route may be both direct and indirect inhalation of infectious droplets or through contact with contaminated fomites',
 'it can be transmitted via the fecal - oral / ocular route',
 'it can be transmitted via the fecal - oral / ocular route',
 'air',
 'air',
 'droplets',
 'droplets',
 'viruses are often transmitted via food and the environment',
 'the virus has high transmissibility and is rapidly transmitted to people through close contact and droplets from coughing , sneezing and talking loudly , as well as through contact with contaminated objects']

In [ ]:
#contexts = list(set(contexts))
contexts

['Enteric viruses are those human viruses that are primarily transmitted by the fecal-oral route, either by person-to-person contact or by ingestion of contaminated food or water. The importance of viral foodborne diseases is increasingly being recognized, and several international organizations have found that there is an upward trend in their incidence. Thus, in this review, state-of-the-art information regarding virus persistence in food and the environment is compiled.',
 'Pathogenic microorganisms usually originate from an infected host or directly from the environment; however, only a small proportion of these microbes cause infection. This chapter discusses the pathogens that are transmitted through the environment such as bacteria, viruses, and protozoa. Many human pathogens can be transmitted only by direct or close contact with an infected person or animal such as herpesvirus. Virus transmission by the airborne route may be both direct and indirect inhalation of infectious dr

In [ ]:
for answer, context in zip(answers,contexts):
  print('answers: ',answers)
  print('/n')
  print('context: ', context)
  print('\n***************************************************************************************\n')

answers:  ['either by person - to - person contact or by ingestion of contaminated food or water', 'virus transmission by the airborne route may be both direct and indirect inhalation of infectious droplets or through contact with contaminated fomites', 'it can be transmitted via the fecal - oral / ocular route', 'it can be transmitted via the fecal - oral / ocular route', 'air', 'air', 'droplets', 'droplets', 'viruses are often transmitted via food and the environment', 'the virus has high transmissibility and is rapidly transmitted to people through close contact and droplets from coughing , sneezing and talking loudly , as well as through contact with contaminated objects']
/n
context:  Enteric viruses are those human viruses that are primarily transmitted by the fecal-oral route, either by person-to-person contact or by ingestion of contaminated food or water. The importance of viral foodborne diseases is increasingly being recognized, and several international organizations have f